In [1]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta
import classes_func
from classes_func import *

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Creating a search instance with Search('City',start_day,end_day)
Barcelona = Search('Barcelona','2024-05-29','2024-06-02')

In [3]:
#Inputing city
Barcelona.input_city()

In [4]:
Barcelona.open_date_box()

In [5]:
#Select Date_start
Barcelona.date_selector(Barcelona.start_day)

In [6]:
#Select Date_end
Barcelona.date_selector(Barcelona.end_day)

In [7]:
#Search for Results
Barcelona.search_results()

In [9]:
#Total number of Result pages
pages = Barcelona.result_pages()
pages

39

In [10]:
#Barcelona.scrape_results(pages)
# Also if it's going to slow, adjust the waiting times in the class_function
Barcelona.scrape_results(5)#Add the number of pages you'll like to scrape

page 1
page 2
page 3
page 4
page 5


In [11]:
Barcelona.df.shape

(116, 8)

In [12]:
Barcelona.scrape_descriptions()

Processing Links:  40%|██████████████████████████████▌                                              | 46/116 [00:31<01:50,  1.58s/it]

Scraped 50 links


Processing Links:  85%|█████████████████████████████████████████████████████████████████▋           | 99/116 [00:53<00:09,  1.86it/s]

Scraped 100 links


Processing Links: 100%|████████████████████████████████████████████████████████████████████████████| 116/116 [01:03<00:00,  1.83it/s]

Scraped 116 links


In [17]:
Barcelona.df.head()

,Hotels,Distance,District,Price,Rating,Stars,Comments,Link,descriptions
0,Sixtytwo Hotel,"a 0,8 km del centro","Eixample, Barcelona",€ 1.513,"9,2",4 de 5,1.833 comentarios,https://www.booking.com/hotel/es/sixtytwo-barc...,"Habitación triple/Ramblas se encuentra a 2,9 k..."
1,INNSiDE by Meliá Barcelona Apolo,"a 1,4 km del centro","Sants-Montjuïc, Barcelona",€ 1.115,"8,9",4 de 5,1.313 comentarios,https://www.booking.com/hotel/es/innside-by-me...,Nice apartment BCN two rooms wifi está en el b...
2,SEALONA VILA OLIMPICA,"a 1,8 km del centro","Sant Martí, Barcelona",€ 360,"8,0",4 de 5,58 comentarios externos,https://www.booking.com/hotel/es/sealona-vila-...,Este hotel moderno ofrece conexión a internet ...
3,Habitaciones Viladomat,"a 1,7 km del centro","Eixample, Barcelona",€ 374,"8,4",4 de 5,NaN,https://www.booking.com/hotel/es/habitaciones-...,"SEALONA VILA OLIMPICA, que cuenta con biciclet..."
4,Hotel Sansi Barcelona,a 450 m del centro,"Eixample, Barcelona",€ 884,"7,2",2 de 5,780 comentarios,https://www.booking.com/hotel/es/sansi-barcelo...,Habitaciones Viladomat se encuentra en Barcelo...


In [16]:
Barcelona.browser.quit()